# 📏 Evaluación de Modelos: Clasificación vs Regresión

Este notebook resume y ejemplifica las métricas más importantes para evaluar modelos de:

- Clasificación (binaria o multiclase)
- Regresión

Usamos ejemplos prácticos con sklearn y matplotlib para visualizar cómo se comportan las métricas.


## Clasificación - Métricas clave

In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Cargar y preparar el dataset
df = sns.load_dataset('titanic')

# Preprocesamiento similar al notebook anterior
df = df.drop(columns=['deck', 'embark_town', 'alive'])
df['age'] = df['age'].fillna(df['age'].median())
df['embarked'] = df['embarked'].fillna(df['embarked'].mode()[0])
df['sex'] = df['sex'].map({'male': 0, 'female': 1})
df = pd.get_dummies(df, columns=['embarked'], drop_first=True)

# Crear nuevas variables
df['family_size'] = df['sibsp'] + df['parch'] + 1
df['is_alone'] = (df['family_size'] == 1).astype(int)

# Escalar
scaler = StandardScaler()
df[['age', 'fare']] = scaler.fit_transform(df[['age', 'fare']])

# Selección de variables
features = ['pclass', 'sex', 'age', 'fare', 'family_size', 'is_alone', 'embarked_Q', 'embarked_S']
X = df[features]
y = df['survived']

# División de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Obtenemos el dataset del Titanic
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, ConfusionMatrixDisplay

# Entrenamiento rápido
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]  # Probabilidades para ROC AUC

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_proba))

In [ ]:
# Matriz de confusión
ConfusionMatrixDisplay.from_estimator(model, X_test, y_test)

### ¿Cuándo usar cada métrica de clasificación?

| Métrica    | ¿Qué mide?                                 | Cuándo usarla                              |
|------------|---------------------------------------------|--------------------------------------------|
| **Accuracy**   | Porcentaje de predicciones correctas                | Cuando las clases están balanceadas              |
| **Precision**  | % de positivos predichos que son correctos | Si quieres evitar falsos positivos (ej: spam) |
| **Recall**     | % de verdaderos positivos capturados       | Si no quieres perder ningún positivo (ej: cáncer) |
| **F1 Score**   | Media armónica de precision y recall       | Si quieres equilibrio entre ambos          |
| **ROC AUC**    | Capacidad del modelo de discriminar entre clases                | Para comparar clasificadores en general |


## Regresión - Métricas clave

Partimos de un nuevo dataset, Boston Housing: fetch_california_housing()

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

data = fetch_california_housing()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))
print("R2 Score:", r2_score(y_test, y_pred))

### ¿Cuándo usar cada métrica de regresión?

| Métrica    | ¿Qué mide?                                  | Cuándo usarla                              |
|------------|----------------------------------------------|--------------------------------------------|
| **MAE**        | Error medio absoluto                         | Fácil de interpretar, útil si te importan los errores medios |
| **RMSE**       | Error cuadrático medio (penaliza errores grandes) | Si quieres penalizar mucho los errores grandes      |
| **R² Score**         | Porcentaje de varianza explicada por el modelo       | Para evaluar calidad general del modelo en su conjunto           |


## Visualización del ajuste (Predicción - Realidad)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Valores reales')
plt.ylabel('Predicciones')
plt.title('Predicciones vs Realidad')
plt.grid(True)
plt.show()

## Conclusión

Elegir la métrica correcta depende del objetivo del problema:

- Clasificación médica: **Recall** > Precision
- Detección de fraude: **Precision** > Recall
- Clasificación general: **F1 Score**
- Comparar modelos: **ROC AUC**

En regresión:
- MAE si te importan errores medios
- RMSE si penalizas mucho los errores grandes
- R² si quieres ver cuánta variabilidad estás explicando
